<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TADBit-HiC-processing-parameters-and-commands" data-toc-modified-id="TADBit-HiC-processing-parameters-and-commands-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TADBit HiC processing parameters and commands</a></span></li><li><span><a href="#TADBit-merging-7-replicas" data-toc-modified-id="TADBit-merging-7-replicas-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>TADBit merging 7 replicas</a></span><ul class="toc-item"><li><span><a href="#Change-to-HOMER-format" data-toc-modified-id="Change-to-HOMER-format-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Change to HOMER format</a></span></li></ul></li><li><span><a href="#HOMER-processing" data-toc-modified-id="HOMER-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>HOMER processing</a></span></li><li><span><a href="#Overlapping-the-significant-interactions-with-SNPs" data-toc-modified-id="Overlapping-the-significant-interactions-with-SNPs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Overlapping the significant interactions with SNPs</a></span></li></ul></div>

**With this script you can process the HiC data (from valid pairs) and overlap it with our 624 LMI selected SNPs to get their interacting counter parts.**

### TADBit HiC processing parameters and commands

We processed our 7 replicas from pancreas HiC experiment using TADbit (Serra *et al.,* PLoS Comp Bio, 2017) to obtain one file with the valid pair read interactions for each replica.

Start here below processing the valid pairs from your samples:

### TADBit merging 7 replicas

In [ ]:
# Count the valid pairs at each of the replicas
parallel wc -l /home/jrodriguez/scratch/pancreas_HiC/pancreas{}/03_filtered_reads/valid* ::: {1..7}

2269506 /home/jrodriguez/scratch/pancreas_HiC/pancreas5/03_filtered_reads/valid_r1-r2_intersection_e7dd1ea461.tsv
 
2578229 /home/jrodriguez/scratch/pancreas_HiC/pancreas4/03_filtered_reads/valid_r1-r2_intersection_e7dd1ea461.tsv

3759181 /home/jrodriguez/scratch/pancreas_HiC/pancreas6/03_filtered_reads/valid_r1-r2_intersection_e7dd1ea461.tsv 

5127448 /home/jrodriguez/scratch/pancreas_HiC/pancreas3/03_filtered_reads/valid_r1-r2_intersection_e7dd1ea461.tsv

19815943 /home/jrodriguez/scratch/pancreas_HiC/pancreas7/03_filtered_reads/valid_r1-r2_intersection_e7dd1ea461.tsv

24297763 /home/jrodriguez/scratch/pancreas_HiC/pancreas1/03_filtered_reads/valid_r1-r2_intersection_e7dd1ea461.tsv

41226187 /home/jrodriguez/scratch/pancreas_HiC/pancreas2/03_filtered_reads/valid_r1-r2_intersection_e7dd1ea461.tsv

**All summed up result in 99,074,257**
Each valid pairs files has a header of 25 lines. 25x7=175

99,074,257 - 175 = **99,074,082** valid pairs, which correspond to the number seen after the merging of the 7 replicas

In [ ]:
# Concatenate the seven replicas
cat {YOUR_PATH}/pancreas{1..7}/03*/valid* | grep -v '^#' > valid_r1-r2_7replicas.tsv

#### Change to HOMER format

In [ ]:
# Change TADbit valid pairs file to HOMER format
awk -v OFS=$'\t' '{print $1,$2,$3,$4,$8,$9,$10}' valid_r1-r2_7replicas.tsv \| 
awk -F"\t" '{ $4 = ($4 == "0" ? "-" : "+") } 1' OFS="\t" \| 
awk -F"\t" '{ $7 = ($7 == "0" ? "-" : "+") } 1' OFS="\t" > valid_r1-r2_7replicas_HOMER.tsv

### HOMER processing

create a tag directory first

In [ ]:
makeTagDirectory {YOUR_PATH}/hic_data_healthy_homer # name of your to-be-created tag directory
    -format HiCsummary # format of the interactions files 
    valid_r1-r2_7replicas_HOMER.tsv # your file

create a background model

In [ ]:
analyzeHiC {YOUR_PATH}hic_data_healthy_homer 
    -res 40000 # resolution
    -bgonly # generate a background model
    -cpu 24

and finally get the significant interactions 

In [ ]:
analyzeHiC {YOUR_PATH}hic_data_healthy_homer 
    -res 40000 
    -interactions significantInteractions_40000_pancreas_17_new # significant interactions output
    -center # centering the interactions based on read average mapping
    -maxDist 2000000 # maximum distance to find interactions
    -nomatrix 
    -cpu 24

We get a total of `41,832` significant interactions. 

By default, `HOMER` reports interactions with pv <= 0.001. Each interaction has its pv based on a test and is reported on the file that we get after the command above. We decided that we could further filter that output, setting the threshold based on Bonferroni correction, estimated from the number of tests performed. *(See main text for a more detailed explanation)* 

In [ ]:
log(1e-05) # = -11.51293 (This is an R command)
# Filter the Z-score LogP field by those that have lower than -11.51293
logp=-11.51293
# Filtering the interactions obtained
awk -F'\t' -v b=${logp} ' $18 <= b ' significantInteractions_40000_pancreas_17_new > significantInteractions_40000_pancreas_17_new_binp1e5

### Overlapping the significant interactions with SNPs 

Now we select the both sides of the interactions...

In [ ]:
awk -F'\t' -v OFS='\t' '{print $3,$4,$5,$1}' significantInteractions_40000_pancreas_17_new_binp1e5 > interactions_40000_pancreas_17_site1_binp1e5.bed
awk -F'\t' -v OFS='\t' '{print $9,$10,$11,$1}' significantInteractions_40000_pancreas_17_new_binp1e5 > interactions_40000_pancreas_17_site2_binp1e5.bed

Run these commands to get the overlap of the `624 GWAS-LMI SNPs` with the interactions

In [6]:
input="LMI_pv.snps.bed" # the file with the GWAS-LMI SNPs
binp="1e5"
mod="LMI_pv_snp_new"
cd {YOUR_PATH}/go_analysis_with_new_interactions/GO_snp ; 

In [ ]:
# Interactions
intersectBed -a ../interactions_40000_pancreas_17_site2_binp1e5.bed -b ${input} -wa -wb | cut -f4 > site2_${mod}_binp${binp}.interactions
intersectBed -a ../interactions_40000_pancreas_17_site1_binp1e5.bed -b ${input} -wa  -wb | cut -f4 > site1_${mod}_binp${binp}.interactions

In [7]:
direct_bins=($(grep -Fwf site1_${mod}_binp${binp}.interactions ../significantInteractions_40000_pancreas_17_new_binp1e5 | awk -F'\t' -v OFS=$'\t' '{print $1,$2,$3,$4,$5,$6}';
               grep -Fwf site2_${mod}_binp${binp}.interactions ../significantInteractions_40000_pancreas_17_new_binp1e5 | awk -F'\t' -v OFS=$'\t' '{print $1,$8,$9,$10,$11,$12}'))
printf "%s\t%s\t%s\t%s\t%s\t%s\n" "${direct_bins[@]}" | sort -k1 > GWAS_bait_${mod}_bins_binp${binp}

interacting_bins=($(grep -Fwf site1_${mod}_binp${binp}.interactions ../significantInteractions_40000_pancreas_17_new_binp1e5 | awk -F'\t' -v OFS=$'\t' '{print $1,$8,$9,$10,$11,$12}' ; 
                    grep -Fwf site2_${mod}_binp${binp}.interactions ../significantInteractions_40000_pancreas_17_new_binp1e5 | awk -F'\t' -v OFS=$'\t' '{print $1,$2,$3,$4,$5,$6}'))
printf "%s\t%s\t%s\t%s\t%s\t%s\n" "${interacting_bins[@]}" | sort -k1 > GWAS_target_${mod}_bins_binp${binp}

In [8]:
## Identify and remove from the target list those interactions in which bait and target are overlapping a same bin.
## This will lead to targets (3D genomics) not identifying anything different 
## from the standard approach.
## First, identifies them and then excludes them from the target list
grep -vf <(comm -12 \
         <(cut -f2 GWAS_bait_LMI_pv_snp_new_bins_binp1e5 | sort | uniq ) \ 
         <(cut -f2 GWAS_target_LMI_pv_snp_new_bins_binp1e5 | sort | uniq)) \
         GWAS_target_LMI_pv_snp_new_bins_binp1e5 > GWAS_target_LMI_pv_snp_new_bins_binp1e5.NoOverlapBaitTarget

In [ ]:
# change the format of the baits file to a bed to intersect with the SNPs
awk '{print}' GWAS_bait_${mod}_bins_binp${binp} > bait_overlap_LMI_pv_snps.bed

#### TO FINISH THE AWK COMMAND TO SELECT FIELDS 

In [12]:
# identifies the interactions that overlap SNPs on both sides
comm -13 <(cut -f1 GWAS_target_LMI_pv_snp_new_bins_binp1e5.NoOverlapBaitTarget | sort | uniq) <(cut -f1 GWAS_target_LMI_pv_snp_new_bins_binp1e5 | sort | uniq) > both_bait_and_target_overlap.ints 
# removing the interactions we do not want
grep -vf both_bait_and_target_overlap.ints <(intersectBed -a bait_overlap_LMI_pv_snps.bed -b LMI_pv.snps.bed -wa -wb | sort -k4,4) | cut -f5- | sort | uniq > 76_bait_regions.SNPs


**Create a combined table with baits, targets, snps and genes.**

The baits: 

*(is the same that we just got `76_bait_regions.SNPs`, but with more fields.)*

In [15]:
grep -vf both_bait_and_target_overlap.ints \ 
<(intersectBed -a bait_overlap_LMI_pv_snps.bed -b LMI_pv.snps.bed -wa -wb | sort -k4,4) > bait_overlaps.snps

14	53952691	53992691	interaction1007	14	53980388	53980389	rs1959834
18	76361162	76401162	interaction1267	18	76392789	76392790	rs2007105
11	29314259	29354259	interaction1371	11	29353459	29353460	rs12226886
14	53916156	53956156	interaction1422	14	53920178	53920179	rs11849931
14	53916156	53956156	interaction1422	14	53924495	53924496	rs11850783
14	53916156	53956156	interaction1422	14	53942112	53942113	rs7153908
20	32436028	32476028	interaction1448	20	32456862	32456863	rs62209634
22	28477385	28517385	interaction1478	22	28502867	28502868	chr22_28502867_A_C
22	28477385	28517385	interaction1478	22	28504183	28504184	chr22_28504183_A_G
22	28477385	28517385	interaction1478	22	28504258	28504259	chr22_28504258_A_G
grep: write error: Broken pipe


Now we need to cross this file above, which contains all the SNPs that overlap a bait counterpart, repeated as many times as SNPs are overlaping each interaction, with the file of individual targets, see below:

In [16]:
# File with target for all these. Just checking :)
head -n10 GWAS_target_LMI_pv_snp_new_bins_binp1e5.NoOverlapBaitTarget

interaction1007	14-53520000	14	53513149	53553149	+
interaction1267	18-76680000	18	76676433	76716433	+
interaction1371	11-27680000	11	27683551	27723551	+
interaction1422	14-53520000	14	53513872	53553872	+
interaction1448	20-32400000	20	32407377	32447377	+
interaction1478	22-28240000	22	28237254	28277254	+
interaction1581	12-132920000	12	132924371	132964371	+
interaction1778	18-30600000	18	30604852	30644852	+
interaction1982	9-108520000	9	108521161	108561161	+
interaction2480	20-32280000	20	32282049	32322049	+


In [ ]:
# Join the files
awk -v OFS=$'\t' 'NR==FNR {a[$1]=$0; next } $4 in a { print $0, a[$4]}' \ 
GWAS_target_LMI_pv_snp_new_bins_binp1e5.NoOverlapBaitTarget bait_overlaps.snps > all_baits_and_targets.txt

**## END ## **